In [1]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import DemoCodingVsIntergenomicSeqs
from utils import simple_coll_factory, padding_coll_factory, LetterTokenizer, build_vocab
from cnn_model import CNN

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## Choose the dataset

In [3]:
get_dataset_fn = DemoCodingVsIntergenomicSeqs
train_dset = get_dataset_fn('train', force_download=False)

Reference /home/jupyter/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.dna.toplevel.fa.gz already exists. Skipping.
Reference /home/jupyter/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.cdna.all.fa.gz already exists. Skipping.


/home/jupyter/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:87: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/190000 [00:00<?, ?it/s]

## Tokenizer and vocab

In [4]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=False)

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 7
{'C': 4, 'A': 3, '<eos>': 5, 'G': 2, '<pad>': 6, 'T': 1, '<bos>': 0}


## Batch preparation with collate

In [6]:
collate = simple_coll_factory(vocabulary, tokenizer)
train_loader = DataLoader(train_dset, batch_size=32, shuffle=True, collate_fn=collate)

input_len  200


## Model

In [ ]:
max_seq_len = max([len(train_dset[i][0]) for i in range(len(train_dset))])
print("max_seq_len ", max_seq_len)
# Count in added in tokenizer '<bos>' and '<eos>' 
nn_input_len = max_seq_len+2

model = CNN(
    number_of_classes=1,
    vocab_size=vocabulary.__len__(),
    embedding_dim=100,
    input_len=nn_input_len
).cuda()

## Training

In [8]:
model.train(train_loader, epochs=1)
# model(torch.zeros([32, nn_input_len], dtype=torch.long).to('cuda'))

flatten size  76
Epoch 0


/opt/conda/envs/bench_env/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train metrics: 
 Accuracy: 85.1%, Avg loss: 0.569151 



## Testing

In [9]:
test_dset = get_dataset_fn('test', force_download=False)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=True, collate_fn=collate)

model.test(test_loader)

Reference /home/jupyter/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.dna.toplevel.fa.gz already exists. Skipping.
Reference /home/jupyter/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.cdna.all.fa.gz already exists. Skipping.


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/190000 [00:00<?, ?it/s]

test_loss  446.34155571460724
num_batches 782
correct 21198
size 25000
Test Error: 
 Accuracy: 84.8%, Avg loss: 0.570769 

